In [1]:
import joblib
model = joblib.load('../model.pkl')

In [2]:
import pandas as pd
import json
X_val = pd.read_csv('X_val.csv', index_col=0)
y_val = pd.read_csv('y_val.csv', index_col=0)
test = pd.concat([X_val, y_val], axis=1)
bookmaker_odds = json.load(open('odds.json'))

In [3]:
print(model.best_estimator_.predict_proba(test.iloc[0].drop(['Home','Visitor','Date'])))
model.predict_proba(test.iloc[0].drop(['Home','Visitor','Date']))

[0.3994215 0.6005785]


array([0.3994215, 0.6005785])

In [4]:
def get_bookmakers_odds(game):
    if game in bookmaker_odds:
        return bookmaker_odds[game]
    return {}

In [5]:
import strategy
from importlib import reload

In [7]:
reload(strategy)
bookmakers = ['Open_Line', 'Pinnacle', '5dimes', 'Heritage', 'Bovada', 'Betonline', 'Average_Line', 'Best_Line', 'Worst_Line']
winnings = dict((bookmaker, 0) for bookmaker in bookmakers)
total_bets = dict((bookmaker, 0) for bookmaker in bookmakers)
possible_bets = dict((bookmaker, 0) for bookmaker in bookmakers)
sum_of_stakes = dict((bookmaker, 0) for bookmaker in bookmakers)
bookmaker_accuracy = dict((bookmaker, 0) for bookmaker in bookmakers)
model_accuracy = 0
total_predictions = 0

stake = 100
threshold = .34
for game in test.iloc:
    game_name = game.Date + game.Visitor + '-' + game.Home
    bookmakers_odds = get_bookmakers_odds(game_name)
    predictions = model.predict_proba(game.drop(['Date','Home','Visitor']))
    for bookmaker, odds in bookmakers_odds.items():
        bet = strategy.bet(stake, predictions, odds, threshold)
        winnings[bookmaker] += bet[game.Winner] / odds[game.Winner] - sum(bet)
        total_bets[bookmaker] += sum(bet) != 0
        possible_bets[bookmaker] += 1
        sum_of_stakes[bookmaker] += sum(bet)
        bookmaker_accuracy[bookmaker] += (1 - round(odds[0])) == game.Winner
    model_accuracy += (1 - round(predictions[0])) == game.Winner
    total_predictions += 1

average_stake = dict((bookmaker, sum_of_stakes[bookmaker] / total_bets[bookmaker]) for bookmaker in bookmakers)
win_per_game = dict((bookmaker, winnings[bookmaker] / total_bets[bookmaker] / (average_stake[bookmaker] / stake)) for bookmaker in bookmakers)
print('Total winnings:\n', winnings, '\n')
print('Average Stake:\n', dict((bookmaker, sum_of_stakes[bookmaker] / total_bets[bookmaker]) for bookmaker in bookmakers))
print('Winning per one betted game:\n', win_per_game, '\n')
print('Average win per one betted game:', sum(list(win_per_game.values())[:6]) / 6)
print('Winnings per 100 games:\n', dict((bookmaker, winnings[bookmaker] / total_bets[bookmaker] * 100) for bookmaker in bookmakers), '\n')
print('The percentage of betted games:\n', dict((bookmaker, total_bets[bookmaker] / possible_bets[bookmaker]) for bookmaker in bookmakers), '\n')
print('Total amount of possible bets:\n', possible_bets, '\n')
print('Bookmaker accuracy\n:', dict((bookmaker, bookmaker_accuracy[bookmaker] / total_bets[bookmaker]) for bookmaker in bookmakers), '\n')
print('Model accuracy:', model_accuracy / total_predictions)

Total winnings:
 {'Open_Line': 10983.242999701944, 'Pinnacle': -2130.8190163020845, '5dimes': -2225.8099217453523, 'Heritage': -364.85565162103154, 'Bovada': -1914.5628350622428, 'Betonline': 2267.7937914350587, 'Average_Line': 125610.5011082534, 'Best_Line': 2393.1612093197177, 'Worst_Line': -5922.646166706438} 

Average Stake:
 {'Open_Line': 117.34089658782706, 'Pinnacle': 124.09481629421064, '5dimes': 124.25894687011836, 'Heritage': 123.95920249726474, 'Bovada': 110.20290524481311, 'Betonline': 120.77548106294861, 'Average_Line': 129.46254880742404, 'Best_Line': 127.25438399499502, 'Worst_Line': 117.7882841069046}
Winning per one betted game:
 {'Open_Line': 10.576401261917136, 'Pinnacle': -1.8766005074068177, '5dimes': -1.9407013342318098, 'Heritage': -0.33034261346465393, 'Bovada': -2.1369090327097005, 'Betonline': 2.10032870165566, 'Average_Line': 106.97308192611115, 'Best_Line': 1.9508423182580683, 'Worst_Line': -5.83996898895667} 

Average win per one betted game: 1.065362745959

<module 'strategy' from '/home/nigga/model-lab/strategy.py'>